**Import data**

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas options
pd.set_option('display.max_colwidth', 1000,'display.max_rows',None, 'display.max_columns',None)

# Plot options
%matplotlib inline
mpl.style.use('ggplot')
sns.set(style = 'whitegrid')


In [7]:
import os
os.listdir(os.getcwd())

['LendingLoanEDA.ipynb',
 'README.md',
 '.ipynb_checkpoints',
 '.git',
 'accepted_2007_to_2018Q4.csv.gz']

In [17]:
loans = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', compression ='gzip',low_memory=True)

/var/folders/8f/c3wp81s97mx_4cyjt0ks4rc80000gn/T/ipykernel_46483/393940706.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', compression ='gzip',low_memory=True)


Check basic dataframe info:

In [8]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Columns: 151 entries, id to settlement_term
dtypes: float64(113), object(38)
memory usage: 2.5+ GB


There are 2.3 million rows and 151 variables. The size of the dataset is 2.5+GB

In [9]:
loans.sample(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
598036,114023483,NaN,12000.0,12000.0,12000.0,36 months,13.59,407.75,C,C2,Clinical Director,3 years,OWN,125000.0,Source Verified,Aug-2017,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=114023483,NaN,home_improvement,Home improvement,373xx,TN,7.81,3.0,Nov-2001,680.0,684.0,2.0,7.0,NaN,11.0,0.0,807.0,12.0,31.0,w,6273.02,6273.02,7738.190000,7738.19,5726.98,2011.21,0.0,0.0,0.0,Mar-2019,407.75,Apr-2019,Mar-2019,714.0,710.0,0.0,26.0,1.0,Individual,NaN,NaN,NaN,0.0,0.0,18235.0,3.0,3.0,1.0,2.0,2.0,17428.0,61.0,2.0,2.0,774.0,52.0,6700.0,0.0,2.0,2.0,4.0,2026.0,226.0,77.4,0.0,0.0,176.0,188.0,5.0,2.0,0.0,5.0,26.0,2.0,18.0,3.0,1.0,2.0,2.0,7.0,15.0,8.0,15.0,2.0,11.0,0.0,0.0,0.0,3.0,76.9,100.0,0.0,0.0,35346.0,18235.0,1000.0,28646.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1805710,6909113,NaN,25975.0,25975.0,25975.0,60 months,25.28,766.68,F,F5,U.S. ARMY,10+ years,MORTGAGE,58000.0,Verified,Sep-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=6909113,NaN,debt_consolidation,Debt consolidation,765xx,TX,22.54,0.0,Mar-2000,670.0,674.0,4.0,NaN,NaN,10.0,0.0,10206.0,49.3,31.0,f,0.00,0.00,34362.499993,34362.50,25975.00,8387.50,0.0,0.0,0.0,Jan-2015,22863.35,NaN,Mar-2019,774.0,770.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,168772.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20700.0,NaN,NaN,NaN,11.0,16877.0,7083.0,55.5,0.0,0.0,164.0,106.0,1.0,1.0,3.0,16.0,NaN,1.0,NaN,0.0,4.0,5.0,5.0,8.0,17.0,6.0,11.0,5.0,10.0,0.0,0.0,0.0,4.0,100.0,40.0,

**Response variable: loan status**

In [10]:
loans['loan_status'].value_counts(dropna=False)

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
NaN                                                         33
Name: loan_status, dtype: int64

For the simplicity of this project, we only consider the loan that is fully paid or charged off.

In [18]:
loans = loans.loc[loans['loan_status'].isin(['Fully Paid','Charged Off'])]

In [19]:
loans.shape # There are 1.3 million loans remaining

(1345310, 151)

In [20]:
loans['loan_status'].value_counts()

Fully Paid     1076751
Charged Off     268559
Name: loan_status, dtype: int64

In [21]:
# status counts as percentage
loans['loan_status'].value_counts(normalize=True)

Fully Paid     0.800374
Charged Off    0.199626
Name: loan_status, dtype: float64

About 80% of the remaining loans have been fully paid and 20% have charged off, so we have a somewhat unbalanced classification problem.

**Feature selection**

The full dataset has 151 features for each loan. We will select features in two steps:
1. Drop features with more than 30% of their data missing
2. Of the remaining features, choose only those that would be available to an investor before deciding to fund the loan.

1. Drop the features missing more than 30% data

In [22]:
# Calculate the percentage of missing data for each feature
missing_fraction = loans.isnull().mean().sort_values(ascending=False)
# Top 10 features missing the most data:
missing_fraction.head(10)

next_pymnt_d                                  1.000000
member_id                                     1.000000
orig_projected_additional_accrued_interest    0.997206
hardship_start_date                           0.995723
hardship_length                               0.995723
hardship_type                                 0.995723
hardship_reason                               0.995723
hardship_status                               0.995723
deferral_term                                 0.995723
hardship_amount                               0.995723
dtype: float64

In [32]:
# store all variables missing more than 30% data:
drop_list = sorted(list(missing_fraction[missing_fraction > 0.3].index))
len(drop_list) # 58 features will be dropped
loans.drop(labels=drop_list, axis = 1, inplace = True)
loans.shape # (1345310, 93)


(1345310, 93)

2. Only keep loan features known to potential investors

In [33]:
print(sorted(loans.columns))

['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'debt_settlement_flag', 'delinq_2yrs', 'delinq_amnt', 'disbursement_method', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'funded_amnt', 'funded_amnt_inv', 'grade', 'hardship_flag', 'home_ownership', 'id', 'initial_list_status', 'inq_last_6mths', 'installment', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'last_pymnt_amnt', 'last_pymnt_d', 'loan_amnt', 'loan_status', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sa